In [2]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

In [5]:
def build_dataset(chorales):
    ragged_chorales = tf.ragged.constant(chorales)
#     print(ragged_chorales)
    dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
    for d in dataset:
        tf.print(d)
        
build_dataset(train_np_list)

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], ..., [66, 61, 57, 54], [66, 61, 57, 54], [66, 61, 57, 54]]
[[64, 59, 55, 52], [64, 59, 55, 52], [64, 59, 55, 52], ..., [71, 63, 54, 47], [71, 63, 54, 47], [71, 63, 54, 47]]
[[69, 65, 62, 50], [69, 65, 62, 50], [69, 65, 62, 50], ..., [62, 57, 54, 50], [62, 57, 54, 50], [62, 57, 54, 50]]
[[62, 57, 53, 50], [62, 57, 53, 50], [62, 57, 53, 50], ..., [62, 57, 54, 38], [62, 57, 54, 38], [62, 57, 54, 38]]
[[67, 62, 59, 55], [67, 62, 59, 55], [67, 62, 59, 54], ..., [67, 62, 59, 43], [67, 62, 59, 43], [67, 62, 59, 43]]
[[74, 71, 67, 55], [74, 71, 67, 55], [74, 71, 67, 55], ..., [67, 62, 59, 43], [67, 62, 59, 43], [67, 62, 59, 43]]
[[74, 69, 65, 62], [74, 69, 65, 62], [74, 69, 65, 62], ..., [74, 66, 57, 50], [74, 66, 57, 50], [74, 66, 57, 50]]
[[69, 64, 61, 45], [69, 64, 61, 45], [69, 64, 62, 47], ..., [69, 64, 61, 45], [69, 64, 61, 45], [69, 64, 61, 45]]
[[69, 64, 61, 57], [69, 64, 61, 57], [69, 64, 61, 57], ..., [69, 64, 61, 45], [69, 64, 6